In [0]:
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [4]:
mnist.train.images.shape

(55000, 784)

In [102]:
28**2

784

In [0]:
n_input=784
input_width=28
input_height=28
n_channel=1

n_conv1=32
n_conv2=64

conv1_k=5
conv2_k=5

n_hidden=1024
n_out=10

pooling_size=2

In [0]:
conv2_width=input_width//(pooling_size*pooling_size)
conv2_height=input_height//(pooling_size*pooling_size)

input_hidden=conv2_width*conv2_height*n_conv2

In [0]:
weights={
    
    'c1':tf.Variable(tf.random_normal([conv1_k,conv1_k,n_channel,n_conv1])),
    'c2':tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    'h1':tf.Variable(tf.random_normal([input_hidden,n_hidden])),
     'out':tf.Variable(tf.random_normal([n_hidden,n_out]))    
    
}

biases={
     'c1':tf.Variable(tf.random_normal([n_conv1])),
     'c2':tf.Variable(tf.random_normal([n_conv2])),
     'h1':tf.Variable(tf.random_normal([n_hidden])),
     'out':tf.Variable(tf.random_normal([n_out]))   
    
    
    
}

In [0]:
def conv2D(x,weights,biases,stride=1):
  conv=tf.nn.conv2d(x,weights,padding="SAME",strides=[1,stride,stride,1])
  out=tf.nn.bias_add(conv,biases)
  out=tf.nn.relu(out)
  return out

def maxpool(x,k=2):
  return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])
  
  

In [0]:
def forward_propogation(x,weights,biases):
  x=tf.reshape(x,shape=[-1,input_width,input_height,n_channel])
  conv1=conv2D(x,weights['c1'],biases['c1'])
  conv1=maxpool(conv1,k=pooling_size)
  
  conv2=conv2D(conv1,weights['c2'],biases['c2'])
  conv2=maxpool(conv2,k=pooling_size)
  
  hidden_input=tf.reshape(conv2,shape=[-1,input_hidden])
  hidden_input=tf.add(tf.matmul(hidden_input,weights['h1']),biases['h1'])
  hidden_out=tf.nn.relu(hidden_input)
  
  out=tf.add(tf.matmul(hidden_out,weights['out']),biases['out'])
  return out
  
  
  
  

In [0]:
x=tf.placeholder("float",[None,n_input])
y=tf.placeholder(tf.int32,[None,n_out])

In [0]:
pred=forward_propogation(x,weights,biases)


In [12]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
optimize_step=optimizer.minimize(cost)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [15]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size) 
        c, _ = sess.run([cost, optimize_step], feed_dict={x:batch_x, y:batch_y})
        total_cost += c
    print(total_cost)

1310453.7567663193
26422.061523719458
17427.844043187797
11797.999267311481
9730.679105597557
10122.393053193344
10539.134844582555
10027.077620137927
8873.864109580225
7902.771047196438
8434.793080472205
6156.071306273341
4971.049176367389
5607.775649548218
4752.904798973679
4165.008540924415
3920.3429651023307
3583.7528607434624
2946.8653512976043
3102.692763412058
3153.7776598634664
2788.4929714234986
2527.9847228310696
2487.214211281182
2269.8241224815474


In [0]:
predictions=tf.argmax(pred,axis=1)
actual_labels=tf.argmax(y,axis=1)
correct_predictions=tf.equal(predictions,actual_labels)
correct=sess.run(correct_predictions,feed_dict={x:mnist.test.images,y:mnist.test.labels})


In [22]:
mnist.test.images.shape[0]

10000

In [23]:
correct.sum()/mnist.test.images.shape[0]

0.9845